In [5]:
from Bio import Entrez
import numpy as np
import pandas as pd
import time

In [6]:
Entrez.email = 'geodynev@gmail.com'

In [11]:
def get_pubmed_count(query):
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmode='xml',
                            retmax=100_000,
                            term=f'"{query}" AND "RNA"')
    
    result = int(Entrez.read(handle)['Count'])
    
    return result

In [13]:
%%time
get_pubmed_count('pseudouridine')

CPU times: total: 62.5 ms
Wall time: 2.05 s


1104

In [17]:
all_mods = pd.read_excel('../data/modomics_all_mods.xlsx')

In [18]:
all_mods

,modomics_code,modification,alias
0,00551A,2'-O-[(5'-phospho)ribosyl]adenosine-5'-monopho...,pAr(p)
1,00551G,2'-O-ribosylguanosine (phosphate)-5'-monophosp...,pGr(p)
2,00A,2'-O-ribosyladenosine (phosphate),Ar(p)
3,00G,2'-O-ribosylguanosine (phosphate),Gr(p)
4,01551A,"1,2'-O-dimethyladenosine-5'-monophosphate",pm1Am
...,...,...,...
319,9551A,inosine-5'-monophosphate,pI
320,9551U,pseudouridine-5'-monophosphate,pY
321,9553N,guanosine triphosphate 5' cap (cap G),GpppN
322,9A,inosine,I


In [19]:
all_mods = all_mods[~all_mods['modification'].str.contains('\(phosphate\)')]

In [20]:
all_mods = all_mods[~all_mods['modification'].str.contains('5\'-monophosphate')]

In [21]:
all_mods = all_mods[~all_mods['modification'].str.contains('5\'-monophospate')]

In [22]:
all_mods

,modomics_code,modification,alias
7,019A,"1,2'-O-dimethylinosine",m1Im
8,01A,"1,2'-O-dimethyladenosine",m1Am
9,01G,"1,2'-O-dimethylguanosine",m1Gm
11,022G,"N2,N2,2'-O-trimethylguanosine","m2,2Gm"
16,027G,"N2,7,2'-O-trimethylguanosine","m2,7Gm"
...,...,...,...
317,8A,8-methyladenosine,m8A
318,8U,dihydrouridine,D
321,9553N,guanosine triphosphate 5' cap (cap G),GpppN
322,9A,inosine,I


In [ ]:
all_mods['pubmed_count'] = all_mods['modification'].apply(get_pubmed_count)

In [33]:
all_mods = all_mods.sort_values(by='pubmed_count', ascending=False)

In [34]:
all_mods.head(25)

,modomics_code,modification,alias,modification_pubmed
204,4155N,5' (3' -dephosphoacetyl-CoA),acCoApN,24716
273,583U,5-(isopentenylaminomethyl)uridine,inm5U,5269
152,2551N,alpha-dimethylmonophosphate 5' cap,mmpN,3630
89,1553N,gamma-methyltriphosphate 5' cap,mpppN,3630
88,1551N,alpha-methylmonophosphate 5' cap,mpN,3630
306,6A,N6-methyladenosine,m6A,2255
196,348G,7-aminocarboxypropylwyosine methyl ester,yW-58,2094
322,9A,inosine,I,1897
323,9U,pseudouridine,Ψ,1101
103,20551U,1-(5-O-phosphono-beta-D-ribofuranosyl)-2-selan...,pse2U,606


In [35]:
all_mods.to_excel('../results/mods_pubmed_initial.xlsx', index=False)